## Aula 2

Objetivo: encontrar uma relação entre Y e X, sendo X as variáveis quantitativas que descrevem Y.

1. Modelização
2. Previsão

Teste estatístico: p-value.

É necessário fazer a decomposição de variáveis para encontrar a parte de Y que se explica por meio de X. Para isso, usa-se a **Analysis of Variance** - ANOVA.


In [ ]:
data(iris)

In [ ]:
# modelo <- aov(variável_dependente ~ variável_independente, data = seu_data_frame)

modelo <- aov(Sepal.Length ~ Species, data = iris)


Para obter a tabela de ANOVA, usamos a função summary():

In [ ]:
summary(modelo)

Antes de interpretar os resultados, é importante verificar se as suposições da ANOVA foram atendidas.

In [ ]:
shapiro.test(modelo$residuals)

## Seleção dos modelos

métodos de seleção de modelos estatísticos, em particular em regressão, onde o objetivo é encontrar o modelo mais adequado, 

### Critério AIC(Critério de Informação de Akaike): −2∗LogLik +2k

O modelo com o menor valor de AIC é considerado o melhor.
O AIC penaliza modelos mais complexos (com mais variáveis) para evitar o sobreajuste.

### BIC (Critério de Informação Bayesiano):

Em grandes amostras, o BIC penaliza muito mais modelos com muitas variáveis do que o AIC, tornando-o mais rigoroso.


In [ ]:
# Ajustando dois modelos
modelo1 <- lm(Sepal.Length ~ Sepal.Width + Petal.Length, data = iris)
modelo2 <- lm(Sepal.Length ~ Sepal.Width + Petal.Length + Petal.Width, data = iris)

# Comparando os modelos com AIC e BIC
aic1 <- AIC(modelo1)
aic2 <- AIC(modelo2)

bic1 <- BIC(modelo1)
bic2 <- BIC(modelo2)

# Exibindo os resultados
cat("AIC do Modelo 1:", aic1, "\n")
cat("AIC do Modelo 2:", aic2, "\n")
cat("BIC do Modelo 1:", bic1, "\n")
cat("BIC do Modelo 2:", bic2, "\n")


### Algoritmos de seleção de variáveis explicativas

O próximo passo é compreender e aplicar os algoritmos de seleção de variáveis explicativas, que ajudam a encontrar o melhor modelo estatístico ao adicionar ou remover variáveis com base em critérios de significância estatística e qualidade do ajuste. A ideia é otimizar o modelo, mantendo apenas as variáveis mais relevantes.

Resumo das Três Métodos de Seleção:

1. Seleção Ascendente (Forward Selection)
- Início: Começa com o modelo mais simples (apenas uma variável ou nenhum regressor).
- Processo: Adiciona progressivamente a variável mais relevante ao modelo.
A variável escolhida é aquela que melhora mais significativamente a qualidade do modelo. Isso pode ser medido por:
- A maior redução do erro residual (Sum of Squares, SS).
- O menor valor de AIC ou BIC.
- A menor p-value associada ao coeficiente da variável.
- Parada: O processo termina quando nenhuma variável restante melhora significativamente o modelo.


In [ ]:
library(MASS)

# Modelo inicial (apenas intercepto)
modelo_inicial <- lm(Sepal.Length ~ 1, data = iris)

# Modelo completo (todas as variáveis)
modelo_completo <- lm(Sepal.Length ~ ., data = iris)

# Forward Selection
modelo_forward <- stepAIC(modelo_inicial, 
                          scope = list(lower = modelo_inicial, upper = modelo_completo), 
                          direction = "forward")

summary(modelo_forward)

2. Seleção Descendente (Backward Selection)
Início: Começa com o modelo completo (todas as variáveis disponíveis).
Processo:
Remove progressivamente a variável menos significativa.
A variável escolhida para remoção é:
Aquela que contribui menos para a soma dos quadrados (Sum of Squares).
Aquela com o maior valor de p-value (menos significativa).
Ou aquela cuja remoção reduz menos o AIC ou BIC.
Parada: O processo termina quando todas as variáveis restantes são estatisticamente significativas.


In [ ]:
# Backward Selection
modelo_backward <- step(modelo_completo, direction = "backward")

summary(modelo_backward)

3. Seleção Passo a Passo (Stepwise Selection)
Combinação de Forward e Backward:
Começa com um modelo simples ou intermediário.
Em cada etapa, adiciona ou remove variáveis, reavaliando o modelo.
Verifica constantemente se a inclusão de novas variáveis torna alguma variável existente irrelevante e permite sua remoção.
Mais complexo, mas garante maior flexibilidade e uma seleção mais ajustada.
Critérios: Como nos outros métodos, a decisão de incluir ou excluir variáveis pode se basear em AIC, BIC, p-values ou outros critérios.

In [ ]:
# Stepwise Selection
modelo_stepwise <- stepAIC(modelo_completo, direction = "both")

summary(modelo_stepwise)

## Aula 3. ACP 

A diferença principal entre os métodos de seleção de variáveis explicativas e a Análise em Componentes Principais (ACP) está no objetivo e na abordagem de cada técnica. Enquanto os métodos de seleção de variáveis explicativas escolhem variáveis individuais relevantes para o modelo, a ACP cria novas variáveis (componentes principais) a partir de combinações lineares das variáveis originais. É uma compressão de dados.

Resumo:

- Reduzir a dimensionalidade do conjunto de dados transformando as variáveis originais em novas variáveis não correlacionadas chamadas componentes principais.
- O objetivo é capturar a maior variância possível em um número reduzido de componentes.
- Utilizada principalmente para simplificação de dados, detecção de padrões, ou visualização.

Um componente principal é uma combinação linear das variáveis originais que captura a maior parte da variância dos dados. Ele é criado de forma a ser não correlacionado com os outros componentes principais e, geralmente, serve para reduzir a dimensionalidade do conjunto de dados.

#### Carregar os dados

In [ ]:
data(iris)

# Selecionar apenas as variáveis numéricas
dados_numericos <- iris[, 1:4]

#### Aplicar análise de componentes principais

In [ ]:
cor(dados_numericos)

Você pode aplicar a normalização caso as unidades sejam distintas:

In [ ]:
# Aplicar a Análise em Componentes Principais
acp <- prcomp(dados_numericos, scale. = TRUE)

# Resumo dos componentes principais
summary(acp)

A saída mostra:

- Proportion of Variance: Quanto da variância total é explicada por cada componente principal.
- Cumulative Proportion: A proporção acumulada de variância explicada.

> Podemos visualizar os componentes principais para entender a importância de cada um!

2. Você pode aplicar somente o PCA:

In [ ]:
library(FactoMineR)

In [ ]:
acp <- PCA(dados_numericos)

# Resumo dos componentes principais
summary(acp)

In [ ]:
# Variância explicada por cada componente
variancia_explicada <- acp$sdev^2 / sum(acp$sdev^2)

# Gráfico de variância explicada
barplot(variancia_explicada,
        main = "Variância Explicada por Componente",
        xlab = "Componentes Principais",
        ylab = "Proporção de Variância",
        names.arg = paste0("PC", 1:4))

Os componentes principais podem ser usados para projetar os dados em um novo espaço com menor dimensionalidade.

In [ ]:
# Projeção dos dados no novo espaço
dados_transformados <- acp$x

# Visualizar as primeiras linhas dos dados transformados
head(dados_transformados)

In [ ]:
# Plotar os dados no espaço dos dois primeiros componentes principais
plot(dados_transformados[, 1], dados_transformados[, 2],
     col = iris$Species, pch = 19,
     xlab = "Componente Principal 1",
     ylab = "Componente Principal 2",
     main = "Visualização no Espaço dos Componentes Principais")
legend("topright", legend = levels(iris$Species), col = 1:3, pch = 19)

## Aula 4. ACF e AFCM

Objetivo: Identificar as estruturas latentes (ou fatores) que explicam as correlações entre as variáveis.
Assume que as variáveis observadas são influenciadas por fatores latentes subjacentes (não observáveis) e por erros específicos.
É mais voltada para entender as relações subjacentes entre variáveis e não necessariamente para reduzir a dimensionalidade.

Teste do Quidrado:

O teste do qui-quadrado é uma técnica estatística usada para verificar associações entre variáveis categóricas ou a adequação de distribuições observadas. Ele é amplamente utilizado em pesquisas para testar hipóteses, especialmente em ciências sociais, biológicas e de mercado. Existem duas principais aplicações do teste do qui-quadrado:

In [ ]:
resultado <- chisq.test(tabela)

## Classe 5 - Classification Automatique

Grupos (ou clusters) identificados pela classificação permitem uma análise específica de cada segmento.
Isso é especialmente útil para tomar decisões práticas, como direcionar campanhas de marketing ou personalizar serviços.


Definições:

- Uma classe ou um cluster: conjunto de objetos que compartilham características similares.
Em um conjunto de dados sobre flores, um cluster pode agrupar flores com pétalas curtas e sépalas largas, enquanto outro cluster agrupa flores com pétalas longas.
- A classificação, ou análise de clusters, é o processo de agrupar um conjunto de objetos em classes ou clusters.

### Etapas típicas em uma análise de clusters

1. Preparação dos dados:

Limpeza dos dados (tratamento de valores ausentes ou extremos).
Normalização das variáveis, para evitar que algumas dominem o resultado devido à escala.
Escolha do algoritmo:

2. Algoritmos comuns:
- k-means: Agrupa os objetos com base na proximidade em um espaço multidimensional.
- Classificação hierárquica (CAH): Cria um dendrograma mostrando relações hierárquicas entre os objetos.

3. Avaliação dos resultados:

Verificar a consistência dos clusters.
Utilizar métricas como o coeficiente de silhueta para avaliar a qualidade dos agrupamentos.

4. Interpretação e visualização:

Entender as características de cada cluster.
Representar os grupos em gráficos, como nuvens de pontos ou dendrogramas.

In [ ]:
# Dados simulados
set.seed(123)
dados <- iris[, 1:4]

# Aplicar k-means (3 clusters)
resultado_kmeans <- kmeans(dados, centers = 3)

# Visualizar os clusters
library(factoextra)
fviz_cluster(resultado_kmeans, data = dados)

Dados binários:

- O índice de Jaccard é ideal para medir similaridade com base apenas nas presenças (1).
- O índice de Sokal e Sneath leva em conta tanto concordâncias positivas (1) quanto negativas (0).



In [ ]:
# Dados dos indivíduos
individuo_i <- c(1, 1, 0, 0, 1, 0)  # Indivíduo 1
individuo_j <- c(1, 0, 1, 0, 1, 0)  # Indivíduo 2

# Tabela de contingência
# Cálculo de a, b, c, d
a <- sum(individuo_i == 1 & individuo_j == 1)
b <- sum(individuo_i == 1 & individuo_j == 0)
c <- sum(individuo_i == 0 & individuo_j == 1)
d <- sum(individuo_i == 0 & individuo_j == 0)

# Exibindo os resultados
cat("a =", a, "| b =", b, "| c =", c, "| d =", d, "\n")

# Cálculo dos índices
jaccard <- a / (a + b + c)
sokal_sneath <- (a + d) / (a + b + c + d)

# Exibindo os resultados
cat("Índice de Jaccard =", jaccard, "\n")
cat("Índice de Sokal e Sneath =", sokal_sneath, "\n")



In [ ]:
# Matriz com dados de múltiplos indivíduos
dados_binarios <- matrix(c(1, 1, 0, 0, 1, 0,
                           1, 0, 1, 0, 1, 0,
                           0, 1, 1, 1, 0, 1), 
                         nrow = 3, byrow = TRUE)

# Nomes dos indivíduos
rownames(dados_binarios) <- c("Indivíduo 1", "Indivíduo 2", "Indivíduo 3")

# Exibir a matriz
print(dados_binarios)


In [ ]:
# Função para calcular os índices de Jaccard e Sokal e Sneath
calcular_indices <- function(v1, v2) {
  a <- sum(v1 == 1 & v2 == 1)
  b <- sum(v1 == 1 & v2 == 0)
  c <- sum(v1 == 0 & v2 == 1)
  d <- sum(v1 == 0 & v2 == 0)
  
  jaccard <- a / (a + b + c)
  sokal_sneath <- (a + d) / (a + b + c + d)
  
  return(c(Jaccard = jaccard, Sokal_Sneath = sokal_sneath))
}

# Aplicar para todos os pares de indivíduos
combinacoes <- combn(1:nrow(dados_binarios), 2)
resultados <- apply(combinacoes, 2, function(par) {
  calcular_indices(dados_binarios[par[1], ], dados_binarios[par[2], ])
})

# Exibir os resultados
rownames(resultados) <- c("Jaccard", "Sokal_Sneath")
colnames(resultados) <- apply(combinacoes, 2, function(par) paste0("Indivíduo ", par[1], " vs ", par[2]))
print(resultados)


### Classificação Hierárquica

Objetivo: Agrupar indivíduos ou objetos em clusters com base em sua similaridade ou distância.

A classificação hierárquica fornece uma estrutura inicial baseada em similaridades globais, enquanto a partição (como k-means) refina os grupos ajustando dinâmicamente os centros.

Métodos comuns:

- Single linkage (distância mínima entre grupos).
- Complete linkage (distância máxima entre grupos).
- Ward (minimiza a variância intragrupo, o mais comum).

Etapa 1: Classificação Hierárquica

In [ ]:
# Dados simulados
set.seed(123)
dados <- matrix(rnorm(100), ncol = 2)

# Cálculo da matriz de distância
distancia <- dist(dados)

# Classificação hierárquica usando o método de Ward
hclust_res <- hclust(distancia, method = "ward.D2")

# Plot do dendrograma
plot(hclust_res, main = "Dendrograma (Método de Ward)")


Etapa 2: Corte do Dendrograma para Obter Clusters

In [ ]:
# Cortar o dendrograma para obter 3 clusters
clusters_iniciais <- cutree(hclust_res, k = 3)

# Visualizar os clusters iniciais
plot(dados, col = clusters_iniciais, pch = 19, main = "Clusters Iniciais da Hierarquia")


Etapa 3: Refinamento com K-Means

In [ ]:
# Usar os clusters iniciais como ponto de partida para k-means
kmeans_res <- kmeans(dados, centers = 3)

# Visualizar os clusters refinados
plot(dados, col = kmeans_res$cluster, pch = 19, main = "Clusters Refinados (K-Means)")
points(kmeans_res$centers, col = 1:3, pch = 8, cex = 2) # Centros dos clusters

Exemplo prático 2:

Passo 1: Análise Fatorial (ACP)
Use a análise em componentes principais para reduzir a dimensionalidade.

In [ ]:
# Carregar dados e biblioteca
data(iris)
library(FactoMineR)

# Realizar a ACP nas variáveis numéricas
acp_res <- PCA(iris[, 1:4], graph = FALSE)

# Ver os componentes principais
print(acp_res$eig)  # Variância explicada pelos componentes


Passo 2: Classificação Hierárquica com Fatores

Realize a classificação hierárquica usando os componentes principais.

In [ ]:
# Usar os dois primeiros componentes principais para a classificação
dados_fatores <- acp_res$ind$coord[, 1:2]  # Coordenadas nos dois primeiros fatores

# Calcular a matriz de distâncias
dist_matrix <- dist(dados_fatores)

# Realizar a classificação hierárquica (método de Ward)
hc_res <- hclust(dist_matrix, method = "ward.D2")

# Plotar o dendrograma
plot(hc_res, main = "Dendrograma usando Fatores da ACP")

Passo 3: Interpretar e Visualizar os Clusters

Projete os clusters obtidos no espaço dos componentes principais.

In [ ]:
print(dist_matrix)

In [ ]:
# Cortar o dendrograma para obter clusters
clusters <- cutree(hc_res, k = 3)

# Plotar os clusters no espaço dos dois primeiros fatores
plot(dados_fatores, col = clusters, pch = 19, 
     xlab = "Componente Principal 1", ylab = "Componente Principal 2",
     main = "Clusters no Espaço dos Fatores")

Se eu quiser classificar um novo indivíduo

In [ ]:
# Exemplo: Novo indivíduo
novo_individuo <- c(5.1, 3.5, 1.4, 0.2)  # Medidas similares a um Setosa

# Calcular distância do novo indivíduo a cada centroide (média dos clusters)
centroides <- aggregate(dados, by = list(Cluster = clusters), FUN = mean)
distancias <- apply(centroides[, -1], 1, function(centroide) {
  sum((novo_individuo - centroide)^2)
})

# Identificar o cluster mais próximo
cluster_novo_individuo <- which.min(distancias)
cat("O novo indivíduo pertence ao cluster:", cluster_novo_individuo, "\n")


Como Saber a Classe de Todos os Indivíduos

In [39]:
table(Cluster = clusters, Species = iris$Species)

       Species
Cluster setosa versicolor virginica
      1     50          0         0
      2      0          9        30
      3      0         41        20

Como saber a classe de um indivíduo único. Por exemplo, o 12ª:

In [38]:
# Classe do indivíduo número 12
classe_individuo_12 <- clusters[12]
cat("O indivíduo 12 pertence ao cluster:", classe_individuo_12, "\n")

# Dados do indivíduo número 12
dados_individuo_12 <- iris[12, ]
cat("Dados do indivíduo 12:\n")
print(dados_individuo_12)


O indivíduo 12 pertence ao cluster: 1 


Dados do indivíduo 12:
   Sepal.Length Sepal.Width Petal.Length Petal.Width Species
12          4.8         3.4          1.6         0.2  setosa


## Curso 6 - Analyse Discriminante

A análise discriminante é uma técnica estatística usada para:

1) Entender se as variáveis independentes podem distinguir os grupos (como em uma análise descritiva).
2) Construir modelos para classificar novas observações nos grupos existentes (como em uma análise preditiva).


Como determinar a discriminação?
Análise de Variância (ANOVA):

Se você tem grupos definidos, use ANOVA para avaliar a variância entre os grupos para cada variável.
Variáveis com maior F-statistic são mais discriminantes.
Coeficientes de Correlação com Classes (Discriminação):

Calcule correlações entre as variáveis quantitativas e as categorias/classes.
Variáveis com maior correlação são mais discriminantes.
Peso nos Componentes Principais (PCA):

Se não houver grupos definidos, use a Análise em Componentes Principais (ACP).
Avalie quais variáveis têm maior peso (loading) nos componentes que explicam a maior parte da variância.
Método de Importância em Modelos:

Use algoritmos de aprendizado supervisionado (ex.: árvores de decisão, regressão logística) para calcular a importância relativa de cada variável.


LDA: Identificar combinações lineares de variáveis que maximizam a separação entre os grupos (análise multivariada). Ela considera a relação entre várias variáveis simultaneamente e fornece: Coeficientes discriminantes, Classificação de indivíduos em grupos, Proporção da variância explicada pelos discriminantes.


ANOVA: Determinar se há diferenças significativas entre as médias de grupos para cada variável (análise univariada).	

In [ ]:
library(MASS)
lda_model <- lda(Performance ~ Runtime + Oxygen_Consumption, data = data)  # Exemplo com variáveis selecionadas
print(lda_model)